In [107]:
import pandas as pd
import os
from dotenv import load_dotenv
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pandas
import openai
from openai import APIError
import os
import json
import re
import numpy as np
from sklearn.cluster import KMeans
from pprint import pprint
from pathlib import Path
import tiktoken
from typing import List, Dict
from sentence_transformers import SentenceTransformer,  models, util
from sentence_transformers.readers import InputExample
from enum import Enum
from torch.utils.data import DataLoader, random_split
from datetime import datetime
import math
import sentence_transformers.losses  as losses
from sentence_transformers import SentenceTransformerTrainingArguments
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction
from sentence_transformers import SentenceTransformerTrainer
from sklearn.model_selection import train_test_split
from datasets import Dataset
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator


In [89]:
def load_df(file: Path) -> pandas.DataFrame:
    loaded_table = pq.read_table(file)
    return loaded_table.to_pandas()

In [90]:
class QueryType(Enum):
    SHORT = 'short_query'
    MEDIUM = 'medium_query'
    LONG = 'long_query'

In [91]:


def convert_dataset(dataframe: pandas.DataFrame, question_type: str) -> List[InputExample]:
    dataset_samples = []
    for _, row in dataframe.iterrows():
        score = float(row['scores'][question_type]) / 5.0
        sample = InputExample(texts=[row['context'], row[question_type]],
                                 label=score)
        dataset_samples.append(sample)
    return dataset_samples

In [92]:
def convert_to_hf_dataset(input_examples):
    # Convert each InputExample into a dictionary
    data_dict = {
        "sentence1": [ex.texts[0] for ex in input_examples],
        "sentence2": [ex.texts[1] for ex in input_examples],
        "score": [ex.label for ex in input_examples]
    }
    
    # Create a Hugging Face Dataset
    return Dataset.from_dict(data_dict)

def get_train_and_eval_datasets():
    df = load_df(file=Path("datasets/train.parquet"))
    training_samples = convert_dataset(df, QueryType.LONG.value)

    # Manually split the dataset while retaining the original structure
    dataset_size = len(training_samples)
    train_size = int(0.8 * dataset_size)

    train_samples = training_samples[:train_size]
    eval_samples = training_samples[train_size:]

    # Convert lists to Hugging Face Datasets
    train_dataset = convert_to_hf_dataset(train_samples)
    eval_dataset = convert_to_hf_dataset(eval_samples)

    return train_dataset, eval_dataset

In [93]:
def make_sentence_transformer(model_name :str) -> SentenceTransformer:
    max_seq_length = 128
    word_embedding_model = models.Transformer(model_name, max_seq_length=max_seq_length)
    # Apply mean pooling to get one fixed sized sentence vector
    """Performs pooling (max or mean) on the token embeddings.
    Iit generates from a variable sized sentence a fixed sized sentence embedding, 
    allows to use the CLS token if it is returned by the underlying word embedding model.
    We can concatenate multiple poolings together.
    - word_embedding_dimension: Dimensions for the word embeddings
    - pooling_mode_cls_token: Use the first token (CLS token) as text representations
    - pooling_mode_max_tokens: Use max in each dimension over all tokens.
    - pooling_mode_mean_tokens: Perform mean-pooling
    """
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                                pooling_mode_cls_token=False,
                                pooling_mode_max_tokens=False,
                                pooling_mode_mean_tokens=True)
    return SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [94]:
model_name = "nesto"
model_save_path = 'output/training_stsbenchmark_'+model_name.replace("/", "-")+'-'+datetime.now().strftime("%d-%m-%Y_%H-%M-%S")

args = SentenceTransformerTrainingArguments(
        # Required parameter:
        output_dir=model_save_path,
        # Optional training parameters:
        num_train_epochs=1,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        learning_rate=2e-5,
        warmup_ratio=0.1,
        fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
        bf16=False,  # Set to True if you have a GPU that supports BF16
        # batch_sampler=BatchSamplers.NO_DUPLICATES,  # losses that use "in-batch negatives" benefit from no duplicates
        # Optional tracking/debugging parameters:
        eval_strategy="steps",
        eval_steps=100,
        save_strategy="steps",
        save_steps=100,
        save_total_limit=2,
        logging_steps=100,
        run_name="proba",  # Will be used in W&B if `wandb` is installed
    )

In [95]:
train_dataset, eval_dataset = get_train_and_eval_datasets()


In [96]:
for s in train_dataset:
    print(s)

{'sentence1': 'Za izvršavanje programa koje pišemo na Pajtonu, potreban nam je program koji se zove Pajton interpreter. Ovaj program tumači (interpretira), a zatim i izvršava Pajton naredbe. Pajton interpreteri mogu da prihvate cele programe i da ih izvrše, a mogu da rade i u interaktivnom režimu, ', 'sentence2': 'Koja je uloga Pajton interpretera u izvršavanju programa na Pajtonu i na koji način tumači Pajton naredbe?', 'score': 0.8}
{'sentence1': 'Sredinom 6. veka, područje naseljavaju Sloveni. U 9. veku, tvrđava Petrikon (ili na slovenskim jezicima - Petrik) ulazi u sastav Bugarskog carstva, a u 11. veku njom vlada sremski vojvoda Sermon, čiji su zlatnici u 19. veku pronađeni u jednom petrovaradinskom vinogradu. Pošto je Bugarska poražena od Vizantije a Sermon ubijen, tvrđava ponovo postaje deo Vizantije, da bi, posle borbi Vizantinaca i Mađara, krajem 12. veka, ušla u sastav srednjovekovne Kraljevine Ugarske. Na području Bačke, ugarska vlast se ustaljuje nešto ranije, tokom 10. vek

In [101]:
def train_a_model(model_name:str, args: SentenceTransformerTrainingArguments):
    num_epochs = 10
    train_dataset, eval_dataset = get_train_and_eval_datasets()
    sentence_transformer = make_sentence_transformer(model_name)
    warmup_steps = math.ceil(len(train_dataset) * num_epochs  * 0.1) #10% of train data for warm-up
    train_loss = losses.CosineSimilarityLoss(model=sentence_transformer)
    train_loss = losses.MatryoshkaLoss(sentence_transformer, train_loss, [768, 512, 256, 128, 64])
  

    sentences1 = [sample['sentence1'] for sample in eval_dataset]
    sentences2 = [sample['sentence2'] for sample in eval_dataset]
    scores = [sample['score'] for sample in eval_dataset]
    print(f'duzina {len(sentences1)}')
    # # 6. (Optional) Create an evaluator & evaluate the base model
    dev_evaluator = EmbeddingSimilarityEvaluator(
        sentences1=sentences1,
        sentences2=sentences2,
        scores=scores,
        main_similarity=SimilarityFunction.COSINE,
        name="sts-dev",
    )

    dev_evaluator(sentence_transformer)

    # 7. Create a trainer & train
    trainer = SentenceTransformerTrainer(
        model=sentence_transformer,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        loss=train_loss,
        evaluator=dev_evaluator,
    )
    trainer.train()

    # # (Optional) Evaluate the trained model on the test set
    test_evaluator = EmbeddingSimilarityEvaluator(
        sentences1=eval_dataset["sentence1"],
        sentences2=eval_dataset["sentence2"],
        scores=eval_dataset["score"],
        main_similarity=SimilarityFunction.COSINE,
        name="sts-dev",
    )
    test_evaluator(sentence_transformer)

    # 8. Save the trained model
    sentence_transformer.save_pretrained("output/mpnet-base-all-nli-triplet/final")

    # 9. (Optional) Push it to the Hugging Face Hub
    # model.push_to_hub("mpnet-base-all-nli-triplet")

In [102]:
train_a_model("google-bert/bert-base-multilingual-cased", args=args)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

duzina 2


  0%|          | 0/1 [00:00<?, ?it/s]

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

{'train_runtime': 6.0678, 'train_samples_per_second': 1.318, 'train_steps_per_second': 0.165, 'train_loss': 0.18574048578739166, 'epoch': 1.0}


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [ ]:
def make_gold_samle(df_train):
    gold_samples = []
    batch_size = 16
    for df in df_train:
        score = float(df['score'])/5.0  # Normalize score to range 0 ... 1
        gold_samples.append(InputExample(texts=[df['sentence1'], df['sentence2']], label=score))
        gold_samples.append(InputExample(texts=[df['sentence2'], df['sentence1']], label=score))
    # We wrap gold_samples (which is a List[InputExample]) into a pytorch DataLoader
    return DataLoader(gold_samples, shuffle=True, batch_size=batch_size)

In [ ]:
cross_encoder_path = 'output/cross-encoder/stsb_indomain_'+model_name.replace("/", "-")+'-'+datetime.now().strftime("%d-%m-%Y_%H-%M-%S")

###### Cross-encoder (simpletransformers) ######
# Use Huggingface/transformers model (like BERT, RoBERTa, XLNet, XLM-R) for cross-encoder model
cross_encoder = CrossEncoder(model_name, num_labels=1)
evaluator = CECorrelationEvaluator.from_input_examples(eval_dataset, name='sts-dev')
train_dataloader = make_gold_samle(train_dataset)
# Configure the training
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up